<a href="https://colab.research.google.com/github/stwater20/AIS3-2024-Material/blob/main/ais3_0x7_upload_event_to_MISP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
!pip3 install pymisp pycryptodome

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 22.0 MB/s eta 0:00:00


In [29]:
from Crypto.Cipher import AES
import base64
import hashlib

def decrypt(encrypted_text, key):
    key = hashlib.sha256(key.encode()).digest()
    encrypted_text = base64.b64decode(encrypted_text)
    nonce = encrypted_text[:16]
    ciphertext = encrypted_text[16:]
    cipher = AES.new(key, AES.MODE_EAX, nonce=nonce)
    plaintext = cipher.decrypt(ciphertext)
    return plaintext.decode('utf-8')


key = "????????"
encrypted_text = "RERfVViq76T7nNS4OSppj3b2PHjrwGMXO7vpMH39MD+s0n2DJajG8aCJjINStFeR1uarAXRvnZg="
decrypted_text = decrypt(encrypted_text, key)


In [2]:
from pymisp import MISPEvent,MISPOrganisation, PyMISP
from dateutil.parser import parser
import json
from pymisp.tools import feed_meta_generator
from io import BytesIO

In [3]:
make_feed = False
misp = None

In [22]:
if make_feed:
    org = MISPOrganisation()
    org.name = "ais3.sectools.tw"
    org.uuid = "d2ded987-c2f1-4b12-bf1d-ba9872d0d452"
else:
    url = "https://misp.sectools.tw"
    key = decrypted_text
    misp = PyMISP(url,key,ssl=False)

/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'misp.sectools.tw'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'misp.sectools.tw'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'misp.sectools.tw'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [27]:
from datetime import datetime
from pathlib import Path
blacklist_ip = ["192.168.1.1"]
for p in blacklist_ip:
    event = MISPEvent()
    event.info = f"[{str(p)}] is suspected of DDoS"
    event.date = datetime.now()
    event.distribution = 3
    event.add_tag('tlp:white')
    if make_feed:
        event.org = org
    else:
        e = misp.search(eventinfo=event.info, metadata=True, pythonify=True)
        if e:
            # Already added.
            continue

    obj_ip = event.add_object(name='domain-ip', standalone=False)
    obj_ip.add_attribute('ip', p)
    obj_ip.add_attribute('first-seen', datetime.now() )
    obj_ip.add_reference(event.uuid, 'related-to',"ais3_friend")

    if make_feed:
        with (Path('output') / f'{event.uuid}.json').open('w') as _w:
            json.dump(event.to_feed(), _w)
    else:
        misp.add_event(event)
    print(event.to_json(indent=2))
if make_feed:
    feed_meta_generator(Path('output'))

/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'misp.sectools.tw'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'misp.sectools.tw'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


{
  "uuid": "47ab1f90-2747-44f4-be81-a97fa5ab5063",
  "Object": [
    {
      "name": "domain-ip",
      "meta-category": "network",
      "template_uuid": "43b3b146-77eb-4931-b4cc-b66c60f28734",
      "description": "A domain/hostname and IP address seen as a tuple in a specific time frame.",
      "template_version": "11",
      "uuid": "cd5dc499-2c64-4a64-beed-a742da612ba8",
      "ObjectReference": [
        {
          "uuid": "4a292914-ac73-40bd-ad5e-2de7d35ddba6",
          "object_uuid": "cd5dc499-2c64-4a64-beed-a742da612ba8",
          "referenced_uuid": "47ab1f90-2747-44f4-be81-a97fa5ab5063",
          "relationship_type": "related-to",
          "comment": "ais3_friend"
        }
      ],
      "Attribute": [
        {
          "uuid": "14cc404e-748d-4cf0-a8a1-ef660d3d92a4",
          "object_relation": "ip",
          "value": "192.168.1.1",
          "type": "ip-dst",
          "category": "Network activity",
          "disable_correlation": false,
          "to_ids": tru